# Torsionsweicher Trägerrost

Das Beispiel des torsionsweichen Trägerrosts zeigt die Anwendung an einem dreidimensionalen Balkentragwerk. Das Kapitel gliedert sich in den Aufgabenbeschrieb, der analytischen Lösung und der Modellierung mit dem NLFE-Modell inklusive dessen Resultaten.

## Aufgabenbeschrieb

Das Beispiel stammt aus @marti_baustatik_2014. Es handelt sich um einen torsionsweichen Trägerrost. In der @fig-trm_uebersicht ist der Grundriss des Trägerrosts dargestellt. Das System besteht aus insgesamt 8 torsionsweichen Stäben, welche an den Enden eingespannt sind.

![Grundriss des torsionsweichen Trägerrosts, nachgezeichnet nach @marti_baustatik_2014](../imgs/trm_uebersicht.svg){#fig-trm_uebersicht}

Es ist die Traglast des Systems zu ermitteln. Die Stablänge und die positiven und negativen Biegewiderstände sind die folgenden.

In [1]:
# | echo: false

from packages import *

In [2]:
# Traglast

# Rastergrösse
l = 1 * un.m  




# Positiver Biegewiderstand
M_u = 100 * un.kN * un.m  

# Negativer Biegewiderstand
M_apos_u = 100 * un.kN * un.m

put_out()

$$
\begin{aligned}
l& = 1 \ \mathrm{m} \quad & M_{u}& = 100 \ \mathrm{kN} \cdot \mathrm{m} \quad & {M}'_{u}& = 100 \ \mathrm{kN} \cdot \mathrm{m} \end{aligned}
$$

## Analytische Lösung

Die analytische Lösung basiert auf dem Traglastverfahren, beschrieben in @marti_baustatik_2014. Mittels eines zulässigen Mechanismus ist ein oberer Grenzwert der Traglast herzuleiten. In der @fig-trm_schnitt sind der innen- und der aussenliegende Stab dargestellt. Aus Symmetriegründen lässt sich der obere Grenzwert des gesamten Trägerrosts anhand dieser Darstellung ermitteln.

![Schnitt des torsionsweichen Trägerrosts mit dem angenommenen Mechanismus für den Innen- und Aussenträger, nachgezeichnet nach @marti_baustatik_2014](../imgs/trm_schnitt.svg){#fig-trm_schnitt}



Die äussere Arbeit $W_a$ des dargestellten Systems in @fig-trm_schnitt beträgt dabei für die am Rand gelegenen Stäbe (Punkte 2'45):

$$
W_{a,2'45} = 4 \cdot \left( Q \cdot \frac{1}{3} + Q \cdot \frac{1}{9} \right)
$$

Und für die Innenträger:

$$
W_{a,123} = 4 \cdot \left( Q \cdot 1 + Q \cdot \frac{1}{3} \right)
$$


Sowie beträgt die innere Arbeit:

$$
W_i = 8 \cdot \left(M_u' + M_u\right) \cdot \left(\frac{1}{3l} +\frac{1}{9l}\right)
$$

Durch das abschliessende Gleichsetzen der Arbeiten und das Lösen nach $Q$ folgt der obere Grenzwert der Traglast zu:

$$
Q = \frac{M_u + M_u'}{2 \cdot l}
$$

Die Plastizitätskontrolle ist in der @fig-trm_schnitt_my gezeigt. Dabei wird eine Lastverteilung von je einer Hälfte $Q$ in $x$ und $y$ Richtung angenommen, welche sich nach vollständigem Umlagern der Biegemomente einstellt.

![Plastizitätskontrolle anhand der Zustandslinien der Biegemomente des Stabausschnitts](../imgs/trm_schnitt_my.svg){#fig-trm_schnitt_my}


Aus der Plastizitätskontrolle geht heraus, dass der Biegewiderstand nirgends überschritten ist. Somit deckt sich der untere und obere Grenzwert der Traglast, sprich die Traglast $Q_u$ ist gefunden. Die analytische Beziehung, mit den numerischen Eingabeparametern substituiert, zeigt die folgende Gleichung:




In [3]:
Q_u = (M_u + M_apos_u) / (2*l)

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
Q_{u}& = \frac{M_{u} + {M}'_{u}}{2 \cdot l} = 100.0 \ \mathrm{kN} \end{aligned}
$$

## NLFE-Modell

Die kompakte analytische Lösung gilt es nun mit dem NLFE-Modell zu approximieren. Das Modell ist in diesem Abschnitt beschrieben. Dazu ist die Systemmodellbildung und die Querschnittsmodellbildung erläutert. Abschliessend sind die Resultate aufgezeigt. 

### Systemmodellbildung

Das System ist in starre Stäbe unterteilt und mit den Gelenken versehen. Die @fig-trm_geom zeigt das statische System. Die Einwirkung ist mit 120 kN modelliert.

![Systemmodell des torsionsweichen Trägerrosts](../imgs/trm_system.svg){#fig-trm_geom}

Ein detaillierter Ausschnitt des Modells zeigt die @fig-trm_detail_system. In dieser sind die Stabanfang- und Endgelenke eingezeichnet, sowie die Elementlänge $l_{El}$ vermasst.

![Detailansicht des Trägerrosts mit Gelenken und vermasster Elementlänge](../imgs/trm_detail_system.svg){#fig-trm_detail_system}

### Querschnittsmodellbildung

Die Querschnittsmodellbildung zeigt die Ermittlung der Biegesteifigkeit und des Abbruchkriteriums auf.

#### Biegesteifigkeit

Die Biegesteifigkeit spielt beim Traglastverfahren eine Nebenrolle, da die Verformungen des Systems von keiner Bedeutung sind. Lediglich der Biegewiderstand ist von Interesse. Das Verhalten lässt sich mit einem ideal-plastischen Momenten-Krümmungs-Diagramm beschreiben. Aufgezeigt in der @fig-trm_m_chi a).

In [4]:
# | echo: false

radians = np.linspace(-0.2, 0.2, 100)
M = np.piecewise(radians, [radians < 0, radians > 0], [-100, 100])

fig, ax = plt.subplots()

ax.plot(radians, M, linestyle="-", color="k")
ax.plot(radians, radians * 0, color="gray")

# Set axis labels
ax.set_xlabel(r"$\varphi$ [rad]")
ax.set_ylabel(r"$M$ [kNm]")

# Set limits
# ax.set_xlim([-0.2, 0.2])
# ax.set_ylim([-101, 101])

# Show the plot
plt.tight_layout()
# plt.savefig("imgs/trm_gelenkdef.svg")
plt.close()

![Ideal-plastisches, a) Momenten-Krümmungs-Diagramm, b) Momenten-Verdrehungs-Diagramm](../imgs/trm_m_chi.svg){#fig-trm_m_chi}


Das ideal-plastische Verhalten ist punktsymmetrisch um den Diagrammursprung. Das Verhalten gilt somit ebenfalls für negative Biegemomente. Die Momenten-Verdrehungs-Beziehung folgt aus der @eq-phi_to_chi_konst. Sämtliche Gelenke weisen die Beziehung gemäss der @fig-trm_m_chi b) auf.


#### Abbruchkriterium

Das Abbruchkriterium vereinfacht sich für das Traglastverfahren. Die Traglast stellt sich bei einer vollständigen Umlagerung sämtlicher Biegemomente ein. Die Traglast ist gefunden, wenn sich die Last nicht mehr steigern lässt. Das Rotationsvermögen ist nicht zu begrenzen.

### Resultate

Mit der Querschnittsmodellbildung ist das System vollständig beschrieben. Es lässt sich analysieren und dessen Resultate ausgeben. Es sind das Last-Verformungs-Verhalten und die entsprechenden Zustandslinien der Biegemomente gezeigt.

![Schematisches Last-Verformungs-Diagramm des Systems. Die auffallenden Steifigkeitsänderungen im System sind mit den entsprechenden Lastparametern markiert](../imgs/trm_q_w.svg){#fig-trm_q_w}

Jeder Knick im Diagrammverlauf beschreibt die Bildung eines plastischen Gelenks. Die Zustandslinien der Biegemomente der entsprechenden Laststufen sind folgend gezeigt. Die @fig-trm_lpa1 zeigt das System bei einem Lastparameter von 0.477. Es bildet sich das erste plastische Gelenk bei den Auflagern der Innenträgern. Die @fig-trm_lpa2 zeigt die Bildung der plastischen Gelenke bei den Auflagern der Aussenträgern. Darauf folgt die Bildung der plastischen Gelenke im Feld der Innenträger, gezeigt in der @fig-trm_lpa3. Abschliessend zeigt die @fig-trm_lpa4 die Bildung der plastischen Gelenke im Feld der Aussenträger. Die Traglast ist damit erreicht, die Last lässt sich nicht weiter steigern. Der Verlauf in der @fig-trm_q_w verdeutlicht dies.

![Qualitative Zustandslinien der Biegemomente des Systems beim Lastparameter 0.477](../imgs/trm_lpa1.svg){#fig-trm_lpa1}


![Qualitative Zustandslinien der Biegemomente des Systems beim Lastparameter 0.747](../imgs/trm_lpa2.svg){#fig-trm_lpa2}


![Qualitative Zustandslinien der Biegemomente des Systems beim Lastparameter 0.779](../imgs/trm_lpa3.svg){#fig-trm_lpa3}



![Qualitative Zustandslinien der Biegemomente des Systems beim Lastparameter 0.833](../imgs/trm_lpa4.svg){#fig-trm_lpa4}

Die Traglast stimmt mit der analytischen Lösung überein. Dies zeigt die folgende Gleichung. Es ist hervorzuheben, dass sich mit dem NLFE-Modell lediglich der untere Grenzwert der Traglast ermitteln lässt. Beim hier gewählten System fallen der obere und der untere Grenzwert jedoch zusammen.


In [5]:
Q_u_NLFE = 0.833 * 120*un.kN

Q_u

put_out(symbolic=True, rows = 1)

$$
\begin{aligned}
Q_{u NLFE}& = 0.833 \cdot 120 = 99.96 \ \mathrm{kN} \\ 
Q_{u}& = \frac{M_{u} + {M}'_{u}}{2 \cdot l} = 100.0 \ \mathrm{kN} \end{aligned}
$$

Das Anwendungsbeispiel zeigt, dass sich dreidimensionale Systeme problemlos modellieren lassen. Die Umlagerung der Biegemomente erfolgt auch im dreidimensionalen Raum. Die numerischen Resultate liegen zuverlässig nahe an der analytischen Lösung. Ausserdem können sämtliche Zustandslinien der Schnittgrössen für jede Laststufe dargestellt werden. Dies erhöht das Verständnis des Tragverhaltens.


In [6]:
DF_trm = pd.read_excel('calc/axis_res.xlsx', sheet_name='trm')

DF_trm['Lpa'] = DF_trm['Lpa']*120
DF_trm['eZ_1 [mm]'] = DF_trm['eZ_1 [mm]']*-1

In [7]:
fig_q_w, ax_q_w = plt.subplots(figsize=(14/2.54,5.4/2.54))


ax_q_w.plot(DF_trm['eZ_1 [mm]'], DF_trm['Lpa'], color='black', linestyle='dashed')

# Add labels and show the plot
ax_q_w.set_ylabel("$q$ [kN/m]")
ax_q_w.set_xlabel("$w$ [mm]")

plt.legend()
# Save the plot to the specified file
# fig_q_w.savefig("imgs/trm_q_w.pdf")
plt.close()

C:\Users\Pascal Gitz\AppData\Local\Temp\ipykernel_7616\51049856.py:10: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
